Import di base

In [1]:
from gym import Env
from gym.spaces import Discrete, Box, MultiDiscrete

from IPython.display import clear_output

import numpy as np
import random
import socket
import time

import struct

In [13]:
# definizioni variabili globali ==================================================================
HOST = "127.0.0.1"
PORT = 7001
PACKET_SIZE = 1024

PacketCounter = 0
PacketCounter_lastCheck = 0

# definizioni funzioni =======================================================

#creo socket e lo faccio partire
def CreaSocket():
    # creo socket server per UE5
    MySocket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

    #Bindo il socket 
    MySocket.bind((HOST,PORT))

    #accendo il socket in ascolto
    MySocket.listen()

    return MySocket


#ciclo di attesa connesione
def AttesaConnesione(MySocket):
    while True:
        RemoteConnection , RemoteAddress = MySocket.accept()
        MySocket.close()
        break
    return RemoteConnection,RemoteAddress

#funzione per sampolare il robot di unreal
def MandaRicevi_Robot(MySocket, R_Start=0.0, R_Arm_1=0.0, R_Arm_2=0.0, R_Arm_3=0.0, R_Arm_4=0.0, R_End=0.0):
    Messaggio =    (struct.pack("f",R_Start)+
                    struct.pack("f",R_Arm_1)+
                    struct.pack("f",R_Arm_2)+
                    struct.pack("f",R_Arm_3)+
                    struct.pack("f",R_Arm_4)+
                    struct.pack("f",R_End)
                    )
    MySocket.sendall(Messaggio)
    ReceivedData = MySocket.recv(PACKET_SIZE)

    return struct.unpack("f",ReceivedData[0:4])[0] , struct.unpack("f",ReceivedData[4:8])[0]

# classe environment di gioco =============================================

class RobotEnv(Env):
    def __init__(self):
        # azioni che possiamo fare
        self.action_space = Discrete(6)
        # Array distanze dal target
        self.observation_space = Box(low=-10000,high=10000,shape=(5,))
        # state:     distanza A, Distanza B, Selettore Target (0-1) , Angolo 1
        self.state = [0.0,0.0,0.0,0.0,0.0]

        self.angoloAttuale_1 = random.uniform(-120,120)
        self.angoloAttuale_2 = random.uniform(0,90)

        self.TargetSelect = random.randint(0,1)

        # lunghezza esercizio
        self.Exercise_length = 1500

    def step(self, action,verbosita=False):
        global UE5_Socket
        
        # applichiamo azione
        
        if action == 0:
            self.angoloAttuale_1 += 2
        elif action == 1:
            self.angoloAttuale_1 -= 2
        elif action == 2:
            pass
        
        if action == 3:
            self.angoloAttuale_2 += 2
        elif action == 4:
            self.angoloAttuale_2 -= 2
        elif action == 5:
            pass
        
        DistA,DistB = MandaRicevi_Robot(UE5_Socket, -90.0, self.angoloAttuale_1, self.angoloAttuale_2)
        
        self.state[4] = self.angoloAttuale_2
        self.state[3] = self.angoloAttuale_1
        self.state[2] = self.TargetSelect
        self.state[1] = DistB
        self.state[0] = DistA
        
        # riduzione tempo 
        self.Exercise_length -= 1

        #calcolo reward
        if self.TargetSelect == 0:
            if self.state[0] <= 40:
                reward = 1
            else:
                reward = -3 * (self.state[0] / 100)
        else:
            if self.state[1] <= 40:
                reward = 1
            else:
                reward = -3 * (self.state[0] / 100)

        done = False

        #verfica se tempo finito
        if self.Exercise_length <= 0:
            done = True
        else:
            done = False
               
        # segnaposto per info
        info = {}
        
        # ritorniamo informazioin sullo step
        return self.state, reward, done, info

    def render(self,mode=0):
        print("Tempo: "+str(self.Exercise_length) + "-- state: " + str(self.state))

    def reset(self):
        # --------------------------------------- TODO target alternato e non random
        #resettiamo temperatura
        self.angoloAttuale_1 = random.uniform(-120,120)
        self.angoloAttuale_2 = random.uniform(0,90)
        self.state = [0.0,0.0,0.0,0.0,0.0]
        self.TargetSelect = random.randint(0,1)
        # resettiamo tempo
        self.Exercise_length = 1500
        return self.state

# codice ==================================================================

#creo e ascolto
Server_Socket = CreaSocket()

#attendo e connetto con UE5 
UE5_Socket , UE5_Address = AttesaConnesione(Server_Socket)

#confermo connesione aperta
print(f"Aperto socket con UE5: {UE5_Address}")

SimpleRobotEnv = RobotEnv()


Aperto socket con UE5: ('127.0.0.1', 56429)


In [15]:
# giro di prova dell'exercise
episodes = 1
for episode in range(1, episodes+1):
    state = SimpleRobotEnv.reset()
    done = False
    score = 0 
    
    while not done:
        SimpleRobotEnv.render(mode=0)
        action = SimpleRobotEnv.action_space.sample()
        n_state, reward, done, info = SimpleRobotEnv.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))

Tempo: 1500-- state: [0.0, 0.0, 0.0, 0.0, 0.0]
Tempo: 1499-- state: [106.62328338623047, 228.9212646484375, 0, 74.23475957949938, 39.71723124657953]
Tempo: 1498-- state: [102.80347442626953, 227.35963439941406, 0, 72.23475957949938, 39.71723124657953]
Tempo: 1497-- state: [106.62328338623047, 228.9212646484375, 0, 74.23475957949938, 39.71723124657953]
Tempo: 1496-- state: [108.23716735839844, 228.9197235107422, 0, 74.23475957949938, 41.71723124657953]
Tempo: 1495-- state: [111.99394226074219, 230.36964416503906, 0, 76.23475957949938, 41.71723124657953]
Tempo: 1494-- state: [113.5455551147461, 230.26800537109375, 0, 76.23475957949938, 43.71723124657953]
Tempo: 1493-- state: [111.99394226074219, 230.36964416503906, 0, 76.23475957949938, 41.71723124657953]
Tempo: 1492-- state: [110.41065216064453, 230.41317749023438, 0, 76.23475957949938, 39.71723124657953]
Tempo: 1491-- state: [111.99394226074219, 230.36964416503906, 0, 76.23475957949938, 41.71723124657953]
Tempo: 1490-- state: [113.5455

In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten
from tensorflow.keras.optimizers import Adam
import tensorflow as tf 

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.experimental.set_virtual_device_configuration(gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024*4)])
  except RuntimeError as e:
    print(e)

states = SimpleRobotEnv.observation_space.shape
actions = SimpleRobotEnv.action_space.n

def build_model(states,actions):
    model = Sequential()
    model.add(Flatten(input_shape=(1,) + SimpleRobotEnv.observation_space.shape))
    model.add(Dense(1024,activation="relu"))
    model.add(Dense(1024,activation="relu"))
    model.add(Dense(1024,activation="relu"))
    model.add(Dense(actions,activation="linear"))
    return model

In [24]:
del model # da usare quando da errore sotto

In [25]:
model = build_model(states,actions)
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_2 (Flatten)          (None, 5)                 0         
_________________________________________________________________
dense_8 (Dense)              (None, 1024)              6144      
_________________________________________________________________
dense_9 (Dense)              (None, 1024)              1049600   
_________________________________________________________________
dense_10 (Dense)             (None, 1024)              1049600   
_________________________________________________________________
dense_11 (Dense)             (None, 6)                 6150      
Total params: 2,111,494
Trainable params: 2,111,494
Non-trainable params: 0
_________________________________________________________________


In [26]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannGumbelQPolicy
from rl.memory import SequentialMemory

def build_agent(model, actions):
    policy = BoltzmannGumbelQPolicy()
    memory = SequentialMemory(limit=50000,window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    
    return dqn

In [29]:
dqn = build_agent(model, actions)
dqn.compile(Adam(learning_rate=1e-3),metrics=["mae"])
dqn.fit(SimpleRobotEnv, nb_steps=500000,visualize=False,verbose=1)

Training for 500000 steps ...
Interval 1 (0 steps performed)


ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host

In [67]:
scores = dqn.test(SimpleRobotEnv, nb_episodes=10,visualize=False)
print(np.mean(scores.history["episode_reward"]))

Testing for 10 episodes ...
Episode 1: reward: 331.749, steps: 350
Episode 2: reward: 27.611, steps: 350
Episode 3: reward: 350.000, steps: 350
Episode 4: reward: 272.813, steps: 350
Episode 5: reward: 350.000, steps: 350
Episode 6: reward: 274.894, steps: 350
Episode 7: reward: 306.500, steps: 350
Episode 8: reward: 130.794, steps: 350
Episode 9: reward: 189.634, steps: 350
Episode 10: reward: 261.353, steps: 350
249.53484972000123


Chiusure varie

In [60]:
model.save("modello_1arm-2target.h5")



In [ ]:
new_model = tf.keras.models.load_model("modello.h5")

In [ ]:
# spengo socket prima di uscire
UE5_Socket.shutdown(socket.SHUT_RDWR)
UE5_Socket.close()

Codice vario utile:

In [ ]:
'''
DistanzaAttuale = MandaRicevi_Robot(
                            UE5_Socket,
                            random.uniform(-230.0,230.0),
                            random.uniform(-230.0,230.0),
                            random.uniform(-230.0,230.0),
                            random.uniform(-230.0,230.0),
                            random.uniform(-230.0,230.0),
                            random.uniform(-230.0,230.0)
                            )
'''

'\nDistanzaAttuale = MandaRicevi_Robot(\n                            UE5_Socket,\n                            random.uniform(-230.0,230.0),\n                            random.uniform(-230.0,230.0),\n                            random.uniform(-230.0,230.0),\n                            random.uniform(-230.0,230.0),\n                            random.uniform(-230.0,230.0),\n                            random.uniform(-230.0,230.0)\n                            )\n'